**Google Colab Connection**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


**Unzipping Dataset Content**

In [ ]:
!unzip "/content/drive/MyDrive/CKplus.zip" -d "/content/dataset"

**Basline Model Training (Change data paths for each dataset**)

In [ ]:
# Multitask Emotion & Gender Classification with CBAM, Fairness, and wandb

import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import LabelEncoder, label_binarize
from itertools import cycle

import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import scipy.ndimage as snd
import matplotlib.pyplot as plt


import wandb

wandb.init(project="multitask-gender-emotion", name="PH-CBAM-CNN-CKplus")

#  Configuration
emotion_labels = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]
gender_labels = ["Man", "Woman"]
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LR = 1e-4

CSV_PATH      = "/content/dataset/CK+/processed/CKPLUS_Features_20250805_111308_faceprefixed_CLEAN.csv"
TRAIN_IMG_DIR = "/content/dataset/CK+/processed/train"
VAL_IMG_DIR   = "/content/dataset/CK+/processed/val"
TEST_IMG_DIR  = "/content/dataset/CK+/processed/test"


MODEL_PATH = "multitask_model_ph_cbam.pt"
PRED_CSV   = "multitask_predictions_ph_cbam.csv"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

emo_le = LabelEncoder().fit(emotion_labels)
gen_le = LabelEncoder().fit(gender_labels)



from torchvision import transforms

train_transform = transforms.Compose([
    transforms.ToPILImage(),

    # Better center-preserving zoom (avoids cutting off facial regions)
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0), ratio=(0.95, 1.05)),

    transforms.RandomHorizontalFlip(p=0.5),      # Standard flip
    transforms.RandomRotation(degrees=10),       # Conservative for CK+ expressions

    # Mild jitter to simulate different lighting environments
    transforms.ColorJitter(
        brightness=0.15,
        contrast=0.15,
        saturation=0.15,
        hue=0.02
    ),

    # Rare but realistic grayscale conversion (retains fine texture)
    transforms.RandomGrayscale(p=0.03),

    # Light blur to help generalize to lower-res datasets
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 0.8)),

    transforms.ToTensor(),

    # Very light erasing (important: aggressive values ruin expression detail)
    transforms.RandomErasing(
        p=0.05,
        scale=(0.01, 0.05),
        ratio=(0.5, 2.0),
        value='random'
    ),

    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

#  PH-CBAM Attention Module
class PH_CBAM(nn.Module):
    def __init__(self, in_channels, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_channels, in_channels // ratio),
            nn.ReLU(),
            nn.Linear(in_channels // ratio, in_channels)
        )
        self.spatial = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Parallel Channel Attention
        avg_out = self.shared_mlp(self.avg_pool(x)).unsqueeze(-1).unsqueeze(-1)
        max_out = self.shared_mlp(self.max_pool(x)).unsqueeze(-1).unsqueeze(-1)
        ch_att = self.sigmoid(avg_out + max_out)
        x_ch = x * ch_att

        # Parallel Spatial Attention
        max_out, _ = torch.max(x_ch, dim=1, keepdim=True)
        avg_out = torch.mean(x_ch, dim=1, keepdim=True)
        sa_input = torch.cat([avg_out, max_out], dim=1)
        sa_att = self.spatial(sa_input)
        return x_ch * sa_att

class AffectNetDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, verbose=True):
        """
        df         : DataFrame slice for the split
        img_dir    : path to images for that split
        transform  : torchvision transform pipeline (or None)
        verbose    : whether to print every 1000th image load
        """
        self.df        = df.reset_index(drop=True)
        self.img_dir   = img_dir
        self.transform = transform
        self.verbose   = verbose
        print(f"CK_plus → {len(self.df)} samples from {self.img_dir}")

    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["filename"])

        if self.verbose and idx % 1000 == 0:
            print(f"[INFO] Loading image {idx}/{len(self.df)}: {img_path}")

        img = cv2.imread(img_path)
        if img is None:
            img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        if self.transform is not None:
            # pass raw ndarray, transform will convert to PIL
            img_tensor = self.transform(img)
        else:
            img = img.astype(np.float32)
            img = (img - img.mean()) / (img.std() + 1e-5)
            img_tensor = torch.tensor(img).permute(2, 0, 1).float()

        emo = torch.tensor(emo_le.transform([row["dominant_emotion"]])[0])
        gen = torch.tensor(gen_le.transform([row["gender"]])[0])

        return img_tensor, emo, gen




#  Multi-Task CNN Model with PH-CBAM
class MultiTaskCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(64), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(128), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(256), nn.MaxPool2d(2),
            PH_CBAM(256)
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(256 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.head_emo = nn.Linear(512, len(emotion_labels))
        self.head_gen = nn.Linear(512, len(gender_labels))

    def forward(self, x):
        x = self.backbone(x)
        x = self.flatten(x)
        x = self.fc(x)
        return self.head_emo(x), self.head_gen(x)

#  Dataloader Factory
def make_loader(df, img_dir, shuffle=True, train=False):
    tfm = train_transform if train else eval_transform
    return DataLoader(
        AffectNetDataset(df, img_dir, transform=tfm),
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        num_workers=0
    )


#  Basic Evaluation Function
def evaluate(model, loader, split_name=""):
    model.eval()
    true_emo, pred_emo, true_gen, pred_gen = [], [], [], []
    with torch.no_grad():
        for batch_idx, (imgs, emo_y, gen_y) in enumerate(tqdm(loader, desc=f"Evaluating {split_name}")):
            imgs = imgs.to(DEVICE)
            emo_y = emo_y.to(DEVICE)
            gen_y = gen_y.to(DEVICE)

            emo_logits, gen_logits = model(imgs)
            true_emo.extend(emo_y.cpu().numpy())
            pred_emo.extend(torch.argmax(emo_logits, dim=1).cpu().numpy())
            true_gen.extend(gen_y.cpu().numpy())
            pred_gen.extend(torch.argmax(gen_logits, dim=1).cpu().numpy())

    df = pd.DataFrame({
        "true_emotion": emo_le.inverse_transform(np.array(true_emo)),
        "pred_emotion": emo_le.inverse_transform(np.array(pred_emo)),
        "true_gender": gen_le.inverse_transform(np.array(true_gen)),
        "pred_gender": gen_le.inverse_transform(np.array(pred_gen)),
    })
    return df

# Confusion Matrix Helper (with percentages)
def plot_confusion(df, title="Confusion"):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    #  Emotion Confusion Matrix
    cm_emo = confusion_matrix(df["true_emotion"], df["pred_emotion"], labels=emotion_labels, normalize='true') * 100
    sns.heatmap(
        cm_emo, annot=True, fmt=".1f", cmap='Blues', ax=axes[0],
        xticklabels=emotion_labels, yticklabels=emotion_labels,
        cbar_kws={'label': 'Percentage (%)'}
    )
    axes[0].set_title(f"{title} - Emotion")
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("True")

    #  Gender Confusion Matrix
    cm_gen = confusion_matrix(df["true_gender"], df["pred_gender"], labels=gender_labels, normalize='true') * 100
    sns.heatmap(
        cm_gen, annot=True, fmt=".1f", cmap='Greens', ax=axes[1],
        xticklabels=gender_labels, yticklabels=gender_labels,
        cbar_kws={'label': 'Percentage (%)'}
    )
    axes[1].set_title(f"{title} - Gender")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("True")

    plt.tight_layout()
    plt.show()
    wandb.log({f"{title}_conf_matrix": wandb.Image(fig)})




# internal helper
def _smooth(arr, win=2, do=True):
    """Uniform 1‑D smoothing for nicer curves."""
    if not do or len(arr) < win:
        return np.asarray(arr)
    return snd.uniform_filter1d(np.asarray(arr, dtype=float), size=win, mode="nearest")

#  Plot Accuracy Curves
def plot_accuracy_curves(train_emo_accs, val_emo_accs,
                         train_gen_accs,  val_gen_accs,
                         smooth=True):
    epochs = np.arange(1, len(train_emo_accs) + 1)
    plt.figure(figsize=(10, 6))

    plt.plot(epochs, _smooth(train_emo_accs, do=smooth), label="Train Emotion Acc",  linewidth=2)
    plt.plot(epochs, _smooth(val_emo_accs,  do=smooth), label="Val   Emotion Acc",  linewidth=2)
    plt.plot(epochs, _smooth(train_gen_accs, do=smooth), "--", label="Train Gender Acc", linewidth=2)
    plt.plot(epochs, _smooth(val_gen_accs,  do=smooth), "--", label="Val   Gender Acc", linewidth=2)

    # mark early‑stop position
    plt.axvline(x=len(epochs), color="grey", linestyle=":", alpha=0.6)

    plt.title("Training vs Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    wandb.log({"accuracy_curves": wandb.Image(plt)})

# Plot Loss Curves
def plot_loss_curves(train_losses, val_losses,
                     train_emo,   val_emo,
                     train_gen,   val_gen,
                     smooth=True):
    epochs = np.arange(1, len(train_losses) + 1)
    plt.figure(figsize=(10, 7))

    plt.plot(epochs, _smooth(train_losses, do=smooth), label="Train Total", linewidth=2)
    plt.plot(epochs, _smooth(val_losses,   do=smooth), label="Val   Total", linewidth=2)

    plt.plot(epochs, _smooth(train_emo, do=smooth), "--", label="Train Emotion")
    plt.plot(epochs, _smooth(val_emo,   do=smooth), "--", label="Val   Emotion")
    plt.plot(epochs, _smooth(train_gen, do=smooth), ":",  label="Train Gender")
    plt.plot(epochs, _smooth(val_gen,   do=smooth), ":",  label="Val   Gender")

    plt.axvline(x=len(epochs), color="grey", linestyle=":", alpha=0.6)

    plt.title("Loss Curves")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    wandb.log({"loss_curves": wandb.Image(plt)})


#  Collect predictions helper
@torch.no_grad()
def collect_preds(loader, return_probs: bool = False):
    model.eval()
    emo_true, emo_pred, gen_true, gen_pred = [], [], [], []
    emo_probs, gen_probs = [], []

    for imgs, e_y, g_y in loader:
        imgs = imgs.to(DEVICE)
        e_y = e_y.to(DEVICE)
        g_y = g_y.to(DEVICE)

        emo_logits, gen_logits = model(imgs)

        emo_true.extend(e_y.cpu().numpy())
        gen_true.extend(g_y.cpu().numpy())
        emo_pred.extend(torch.argmax(emo_logits, 1).cpu().numpy())
        gen_pred.extend(torch.argmax(gen_logits, 1).cpu().numpy())

        if return_probs:
            emo_probs.append(torch.softmax(emo_logits, 1).cpu())
            gen_probs.append(torch.softmax(gen_logits, 1).cpu())

    if return_probs:
        return (
            np.array(emo_true), np.array(emo_pred),
            np.array(gen_true), np.array(gen_pred),
            torch.cat(emo_probs).numpy(), torch.cat(gen_probs).numpy()
        )
    else:
        return (
            np.array(emo_true), np.array(emo_pred),
            np.array(gen_true), np.array(gen_pred)
        )

# Extended ROC Curve Plot
def plot_extended_roc(split_name, true_emo, prob_emo, true_gen, prob_gen):
    true_emo_bin = label_binarize(true_emo, classes=list(range(len(emotion_labels))))
    true_gen_bin = label_binarize(true_gen, classes=list(range(len(gender_labels))))

    fpr_emo = dict()
    tpr_emo = dict()
    roc_auc_emo = dict()

    for i in range(len(emotion_labels)):
        fpr_emo[i], tpr_emo[i], _ = roc_curve(true_emo_bin[:, i], prob_emo[:, i])
        roc_auc_emo[i] = auc(fpr_emo[i], tpr_emo[i])

    fpr_emo["micro"], tpr_emo["micro"], _ = roc_curve(true_emo_bin.ravel(), prob_emo.ravel())
    roc_auc_emo["micro"] = auc(fpr_emo["micro"], tpr_emo["micro"])

    all_fpr = np.unique(np.concatenate([fpr_emo[i] for i in range(len(emotion_labels))]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(len(emotion_labels)):
        mean_tpr += np.interp(all_fpr, fpr_emo[i], tpr_emo[i])
    mean_tpr /= len(emotion_labels)
    fpr_emo["macro"] = all_fpr
    tpr_emo["macro"] = mean_tpr
    roc_auc_emo["macro"] = auc(fpr_emo["macro"], tpr_emo["macro"])

    plt.figure(figsize=(10, 7))
    plt.plot(fpr_emo["micro"], tpr_emo["micro"], linestyle='--', label=f"Emotion Micro-average (AUC = {roc_auc_emo['micro']:.2f})")
    plt.plot(fpr_emo["macro"], tpr_emo["macro"], linestyle='-',  label=f"Emotion Macro-average (AUC = {roc_auc_emo['macro']:.2f})")
    colors = cycle(sns.color_palette("hls", len(emotion_labels)))
    for i, color in zip(range(len(emotion_labels)), colors):
        plt.plot(fpr_emo[i], tpr_emo[i], color=color, label=f"{emotion_labels[i]} (AUC = {roc_auc_emo[i]:.2f})")

    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f"{split_name} Emotion ROC")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right", fontsize="small")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    wandb.log({f"roc_curve/{split_name.lower()}_emotion": wandb.Image(plt)})

    fpr_gen = dict()
    tpr_gen = dict()
    roc_auc_gen = dict()

    for i in range(len(gender_labels)):
        fpr_gen[i], tpr_gen[i], _ = roc_curve(true_gen_bin[:, i], prob_gen[:, i])
        roc_auc_gen[i] = auc(fpr_gen[i], tpr_gen[i])

    fpr_gen["micro"], tpr_gen["micro"], _ = roc_curve(true_gen_bin.ravel(), prob_gen.ravel())
    roc_auc_gen["micro"] = auc(fpr_gen["micro"], tpr_gen["micro"])

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_gen["micro"], tpr_gen["micro"], linestyle='--', label=f"Gender Micro-average (AUC = {roc_auc_gen['micro']:.2f})")
    for i in range(len(gender_labels)):
        plt.plot(fpr_gen[i], tpr_gen[i], label=f"{gender_labels[i]} (AUC = {roc_auc_gen[i]:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f"{split_name} Gender ROC")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    wandb.log({f"roc_curve/{split_name.lower()}_gender": wandb.Image(plt)})

#  Fairness Auditing: Gender × Emotion Accuracy
def fairness_audit(df):
    print("\n Accuracy by Gender × Emotion:")
    accuracy_table = df.groupby(["true_gender", "true_emotion"]).apply(
        lambda g: (g["true_emotion"] == g["pred_emotion"]).mean()
    ).unstack().fillna(0)
    print(accuracy_table.round(3))

    for gender in accuracy_table.index:
        for emotion in accuracy_table.columns:
            wandb.log({f"fairness/{gender}/{emotion}": accuracy_table.loc[gender, emotion]})

    pos_rate = df["pred_gender"].value_counts(normalize=True)
    disp_impact = pos_rate.get("Woman", 0) / (pos_rate.get("Man", 1e-6))
    print(f"\n  Disparate Impact Ratio (Woman / Man): {disp_impact:.3f}")
    wandb.log({"disparate_impact_ratio": disp_impact})

    woman_acc = accuracy_table.loc["Woman"].mean()
    man_acc = accuracy_table.loc["Man"].mean()
    equal_opp_diff = woman_acc - man_acc
    print(f"\n Equal Opportunity Difference (Woman - Man): {equal_opp_diff:.3f}")
    wandb.log({"equal_opportunity_diff": equal_opp_diff})

    plt.figure(figsize=(10, 6))
    accuracy_table.T.plot(kind="bar")
    plt.title("Emotion Classification Accuracy by Gender")
    plt.xlabel("Emotion")
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    plt.grid(axis='y')
    plt.legend(title="Gender")
    plt.tight_layout()
    plt.show()
    wandb.log({"gender_bias_bar_chart": wandb.Image(plt)})

#  Main
if __name__ == "__main__":
    torch.manual_seed(42)
    model = MultiTaskCNN().to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    #Load CSV and Prepare DataLoaders
    df = pd.read_csv(CSV_PATH)

    train_loader = make_loader(
        df[df["dataset"] == "train"],
        TRAIN_IMG_DIR,
        shuffle=True,
        train=True  #  Apply data augmentation
    )

    val_loader = make_loader(
        df[df["dataset"] == "val"],
        VAL_IMG_DIR,
        shuffle=False,
        train=False  # No augmentation
    )

    test_loader = make_loader(
        df[df["dataset"] == "test"],
        TEST_IMG_DIR,
        shuffle=False,
        train=False  #  No augmentation
    )


from torch.optim.lr_scheduler import StepLR

#  Optimizer and Scheduler
initial_lr = 0.001  # Set your desired learning rate here
opt = torch.optim.Adam(model.parameters(), lr=initial_lr)
scheduler = StepLR(opt, step_size=5, gamma=0.5)  # Reduce LR by half every 5 epochs

# === Tracking lists ===
train_losses, val_losses = [], []
train_emo_losses, train_gen_losses = [], []
val_emo_losses, val_gen_losses = [], []

train_emo_accuracies, val_emo_accuracies = [], []
train_gen_accuracies, val_gen_accuracies = [], []

#  Hybrid‑Early‑Stopping Setup
patience          = 5         # Max epochs to wait without improvement
min_delta_loss    = 0.0005        # Minimum drop in loss to count as better
min_delta_acc     = 0.001       # Minimum gain in accuracy to count as better

best_val_loss     = float("inf")
best_val_acc      = 0.0
best_epoch        = 0
epochs_no_improve = 0
#

for epoch in range(EPOCHS):
    print(f"\n📦 Starting Epoch {epoch + 1}/{EPOCHS}")

    # -------- TRAIN --------
    model.train()
    total_loss = total_emo_loss = total_gen_loss = 0.0
    correct_emo = total_emo = correct_gen = total_gen = 0

    loop = tqdm(train_loader, desc=f"🌀 Epoch {epoch+1}/{EPOCHS} [Training]", leave=False)
    for images, emo_labels, gen_labels in loop:
        images, emo_labels, gen_labels = images.to(DEVICE), emo_labels.to(DEVICE), gen_labels.to(DEVICE)

        emo_logits, gen_logits = model(images)
        emo_loss = criterion(emo_logits, emo_labels)
        gen_loss = criterion(gen_logits, gen_labels)
        loss = emo_loss + gen_loss

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss      += loss.item()
        total_emo_loss  += emo_loss.item()
        total_gen_loss  += gen_loss.item()
        correct_emo     += (torch.argmax(emo_logits, 1) == emo_labels).sum().item()
        total_emo       += emo_labels.size(0)
        correct_gen     += (torch.argmax(gen_logits, 1) == gen_labels).sum().item()
        total_gen       += gen_labels.size(0)

    avg_train_loss     = total_loss / len(train_loader)
    avg_emo_loss       = total_emo_loss / len(train_loader)
    avg_gen_loss       = total_gen_loss / len(train_loader)
    train_emo_acc      = correct_emo / total_emo
    train_gen_acc      = correct_gen / total_gen

    train_losses.append(avg_train_loss)
    train_emo_losses.append(avg_emo_loss)
    train_gen_losses.append(avg_gen_loss)
    train_emo_accuracies.append(train_emo_acc)
    train_gen_accuracies.append(train_gen_acc)

    #  VALIDATION
    model.eval()
    val_loss = val_emo_loss = val_gen_loss = 0.0
    val_correct_emo = val_total_emo = val_correct_gen = val_total_gen = 0

    with torch.no_grad():
        for images, emo_labels, gen_labels in val_loader:
            images, emo_labels, gen_labels = images.to(DEVICE), emo_labels.to(DEVICE), gen_labels.to(DEVICE)

            emo_logits, gen_logits = model(images)
            emo_l = criterion(emo_logits, emo_labels)
            gen_l = criterion(gen_logits, gen_labels)

            val_loss      += (emo_l + gen_l).item()
            val_emo_loss  += emo_l.item()
            val_gen_loss  += gen_l.item()
            val_correct_emo += (torch.argmax(emo_logits, 1) == emo_labels).sum().item()
            val_total_emo   += emo_labels.size(0)
            val_correct_gen += (torch.argmax(gen_logits, 1) == gen_labels).sum().item()
            val_total_gen   += gen_labels.size(0)

    avg_val_loss      = val_loss / len(val_loader)
    avg_val_emo_loss  = val_emo_loss / len(val_loader)
    avg_val_gen_loss  = val_gen_loss / len(val_loader)
    val_emo_acc       = val_correct_emo / val_total_emo
    val_gen_acc       = val_correct_gen / val_total_gen

    val_losses.append(avg_val_loss)
    val_emo_losses.append(avg_val_emo_loss)
    val_gen_losses.append(avg_val_gen_loss)
    val_emo_accuracies.append(val_emo_acc)
    val_gen_accuracies.append(val_gen_acc)

    print(
        f" Epoch {epoch+1}: "
        f"Train Loss={avg_train_loss:.4f}, Val Loss={avg_val_loss:.4f}, "
        f"Train Emo Acc={train_emo_acc:.4f}, Val Emo Acc={val_emo_acc:.4f}, "
        f"Train Gen Acc={train_gen_acc:.4f}, Val Gen Acc={val_gen_acc:.4f}"
    )

    # --- Log to Weights & Biases ---
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "val_loss": avg_val_loss,
        "train_emo_loss": avg_emo_loss,
        "val_emo_loss": avg_val_emo_loss,
        "train_gen_loss": avg_gen_loss,
        "val_gen_loss": val_gen_loss,
        "train_emo_accuracy": train_emo_acc,
        "val_emo_accuracy": val_emo_acc,
        "train_gen_accuracy": train_gen_acc,
        "val_gen_accuracy": val_gen_acc,
        "lr": opt.param_groups[0]["lr"],
    })

    # --- Step scheduler ---
    scheduler.step()

    # --- Hybrid Early Stopping ---
    improved_loss = (best_val_loss - avg_val_loss) > min_delta_loss
    improved_acc  = (val_emo_acc - best_val_acc) > min_delta_acc

    if improved_loss or improved_acc:
        epochs_no_improve = 0
        best_epoch = epoch
        if improved_loss:
            best_val_loss = avg_val_loss
        if improved_acc:
            best_val_acc = val_emo_acc
        torch.save(model.state_dict(), MODEL_PATH)
        print(f"📦 New best model saved at epoch {epoch+1} "
              f"(val_loss={best_val_loss:.4f}, val_acc={best_val_acc:.4f})")
    else:
        epochs_no_improve += 1
        print(f"  No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f" Early stopping triggered at epoch {epoch+1} (best epoch was {best_epoch+1})")

        # ─ Trim logs so curves end at best_epoch ─
        end = best_epoch + 1
        train_losses         = train_losses[:end]
        val_losses           = val_losses[:end]
        train_emo_losses     = train_emo_losses[:end]
        val_emo_losses       = val_emo_losses[:end]
        train_gen_losses     = train_gen_losses[:end]
        val_gen_losses       = val_gen_losses[:end]
        train_emo_accuracies = train_emo_accuracies[:end]
        val_emo_accuracies   = val_emo_accuracies[:end]
        train_gen_accuracies = train_gen_accuracies[:end]
        val_gen_accuracies   = val_gen_accuracies[:end]

        break

# END for‑epoch LOOP

# Final Evaluation
print("\n Training complete. Running final evaluation on best model...")
model.load_state_dict(torch.load(MODEL_PATH))

train_df = evaluate(model, train_loader, split_name="train")
val_df   = evaluate(model, val_loader,   split_name="val")
test_df  = evaluate(model, test_loader,  split_name="test")
print(" Evaluation complete.")

# Save test predictions
test_df.to_csv(PRED_CSV, index=False)
print(f" Saved test predictions to {PRED_CSV}")




# Plot confusion matrices
plot_confusion(train_df, title="Train Confusion Matrix")
plot_confusion(val_df,   title="Validation Confusion Matrix")
plot_confusion(test_df,  title="Test Confusion Matrix")

#
#  Classification reports, curves, ROC plots
#
def print_classification_reports(df, split_name=""):
    print(f"\n {split_name} Emotion Classification Report:")
    print(classification_report(df["true_emotion"],
                                df["pred_emotion"],
                                target_names=emotion_labels))

    print(f"\n {split_name} Gender Classification Report:")
    print(classification_report(df["true_gender"],
                                df["pred_gender"],
                                target_names=gender_labels))

print_classification_reports(train_df, "Train")
print_classification_reports(val_df,   "Validation")
print_classification_reports(test_df,  "Test")

# Accuracy / loss curves
plot_accuracy_curves(train_emo_accuracies, val_emo_accuracies,
                     train_gen_accuracies,  val_gen_accuracies)

plot_loss_curves(train_losses, val_losses,
                 train_emo_losses, val_emo_losses,
                 train_gen_losses, val_gen_losses)

#
#  Collect predictions (with probabilities) for ROC curves
#
(
    emo_true_train, _, gen_true_train, _, emo_prob_train, gen_prob_train
) = collect_preds(train_loader, return_probs=True)
(
    emo_true_val,   _, gen_true_val,   _, emo_prob_val,   gen_prob_val
) = collect_preds(val_loader,   return_probs=True)
(
    emo_true_test,  _, gen_true_test,  _, emo_prob_test,  gen_prob_test
) = collect_preds(test_loader,  return_probs=True)

#  ensure gender prob arrays have two columns (binary case)
def _ensure_two_cols(arr):
    """If arr is shape [N] or [N,1], convert to [N,2] as [1-p, p]."""
    arr = np.asarray(arr)
    if arr.ndim == 1 or (arr.ndim == 2 and arr.shape[1] == 1):
        arr = arr.reshape(-1, 1)
        arr = np.hstack([(1.0 - arr), arr])
    return arr

gen_prob_train = _ensure_two_cols(gen_prob_train)
gen_prob_val   = _ensure_two_cols(gen_prob_val)
gen_prob_test  = _ensure_two_cols(gen_prob_test)

#
#  Plot ROC curves
#
plot_extended_roc("Train",      emo_true_train, emo_prob_train,
                  gen_true_train, gen_prob_train)
plot_extended_roc("Validation", emo_true_val,   emo_prob_val,
                  gen_true_val,   gen_prob_val)
plot_extended_roc("Test",       emo_true_test,  emo_prob_test,
                  gen_true_test,  gen_prob_test)

#
#  Fairness Auditing: Gender × Emotion Analysis
#
def fairness_audit(df):
    print("\n Accuracy by Gender × Emotion:")
    accuracy_table = (
        df.groupby(["true_gender", "true_emotion"])
          .apply(lambda g: (g["true_emotion"] == g["pred_emotion"]).mean())
          .unstack()
          .fillna(0)
    )
    print(accuracy_table.round(3))

    # Log each cell to wandb
    for gender in accuracy_table.index:
        for emotion in accuracy_table.columns:
            wandb.log({f"fairness/{gender}/{emotion}": accuracy_table.loc[gender, emotion]})

    #  Disparate Impact
    pos_rate    = df["pred_gender"].value_counts(normalize=True)
    disp_impact = pos_rate.get("Woman", 0) / (pos_rate.get("Man", 1e-6))
    print(f"\n  Disparate Impact Ratio (Woman / Man): {disp_impact:.3f}")
    wandb.log({"disparate_impact_ratio": disp_impact})

    # Equal Opportunity
    woman_acc      = accuracy_table.loc["Woman"].mean()
    man_acc        = accuracy_table.loc["Man"].mean()
    equal_opp_diff = woman_acc - man_acc
    print(f"\n Equal Opportunity Difference (Woman - Man): {equal_opp_diff:.3f}")
    wandb.log({"equal_opportunity_diff": equal_opp_diff})

    #  Bias Visualization Bar Chart
    plt.figure(figsize=(10, 6))
    accuracy_table.T.plot(kind="bar")
    plt.title("Emotion Classification Accuracy by Gender")
    plt.xlabel("Emotion")
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    plt.grid(axis='y')
    plt.legend(title="Gender")
    plt.tight_layout()
    plt.show()
    wandb.log({"gender_bias_bar_chart": wandb.Image(plt)})

#
#  Run the fairness audit on the test set
#
fairness_audit(test_df)



**MobileNet_V3 Model Training (Change data paths for each dataset**)

In [ ]:
#  Multitask Emotion & Gender Classification with MobileNet‑V3, Fairness, and wandb
# (CBAM & custom‑CNN removed – MobileNet V3 Small IMAGENET‑pre‑trained backbone used)

import os, cv2, io, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm
from itertools import cycle
from PIL import Image

from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_curve, auc)
from sklearn.preprocessing import LabelEncoder, label_binarize
import scipy.ndimage as snd

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import MobileNet_V3_Small_Weights as MNV3W
from torch.optim.lr_scheduler import StepLR
from functools import partial


import torch
from torch.utils.data import DataLoader
from torchvision import transforms

import wandb
wandb.init(project="multitask-gender-emotion", name="CKplus-Scratch‑MobileNetV3")

# Config
emotion_labels  = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]
gender_labels   = ["Man", "Woman"]

IMG_SIZE  = 224
BATCH_SIZE = 32
EPOCHS   = 50
LR = 1e-3

CSV_PATH      = "/content/dataset/CK+/processed/CKPLUS_Features_20250805_111308_faceprefixed_CLEAN.csv"
TRAIN_IMG_DIR = "/content/dataset/CK+/processed/train"
VAL_IMG_DIR   = "/content/dataset/CK+/processed/val"
TEST_IMG_DIR  = "/content/dataset/CK+/processed/test"


MODEL_PATH = "multitask_Scratch_model_mnv3.pt"
PRED_CSV   = "multitask_Scratch_predictions_mnv3.csv"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

emo_le = LabelEncoder().fit(emotion_labels)
gen_le = LabelEncoder().fit(gender_labels)





from torchvision import transforms

#  Optimized Train Transform for CK+ Dataset
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.90, 1.0)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(8),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

#  Evaluation Transform (No Augmentation)
eval_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



# Your Dataset Class (unchanged)
class AffectNetDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, verbose=True):
        self.df, self.img_dir, self.transform, self.verbose = df.reset_index(drop=True), img_dir, transform, verbose
        print(f"CKplus → {len(self.df)} samples from {self.img_dir}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row["filename"])

        if self.verbose and idx % 1000 == 0:
            print(f"[INFO] Loading {idx}/{len(self.df)}: {path}")

        img = cv2.imread(path)
        if img is None:
            img = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        img = self.transform(img) if self.transform else transforms.ToTensor()(img / 255.)

        emo = torch.tensor(emo_le.transform([row["dominant_emotion"]])[0])
        gen = torch.tensor(gen_le.transform([row["gender"]])[0])
        return img, emo, gen

#  Your make_loader Function
def make_loader(df, img_dir, *, shuffle, train):
    tfm = train_transform if train else eval_transform
    return DataLoader(
        AffectNetDataset(df, img_dir, transform=tfm),
        batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=0
    )


#  New: Create all loaders
def create_dataloaders(csv_path=CSV_PATH,
                       train_img_dir=TRAIN_IMG_DIR,
                       val_img_dir=VAL_IMG_DIR,
                       test_img_dir=TEST_IMG_DIR,
                       batch_size=BATCH_SIZE):
    # Load full CSV
    df = pd.read_csv(csv_path)

    # Split df by your 'split' column (make sure your CSV has this)
    train_df = df[df['dataset'] == 'train'].reset_index(drop=True)
    val_df = df[df['dataset'] == 'val'].reset_index(drop=True)
    test_df = df[df['dataset'] == 'test'].reset_index(drop=True)

    # Create loaders using your make_loader func and existing transforms
    train_loader = make_loader(train_df, train_img_dir, shuffle=True, train=True)
    val_loader = make_loader(val_df, val_img_dir, shuffle=False, train=False)
    test_loader = make_loader(test_df, test_img_dir, shuffle=False, train=False)

    return train_loader, val_loader, test_loader


def mixup_data(x, y1, y2, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y1_a, y1_b = y1, y1[index]
    y2_a, y2_b = y2, y2[index]
    return mixed_x, y1_a, y1_b, y2_a, y2_b, lam


#  usage
train_loader, val_loader, test_loader = create_dataloaders()


#  Multi‑Task MobileNet‑V3
class MultiTaskCNN(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = models.mobilenet_v3_small(weights=None,
                                              norm_layer=lambda nf: nn.GroupNorm(8, nf))
        self.features = backbone.features
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout1 = nn.Dropout(0.4)   # added before pooling
        self.dropout2 = nn.Dropout(0.5)   # added before FC
        self.head_emo = nn.Linear(576, len(emotion_labels))
        self.head_gen = nn.Linear(576, len(gender_labels))

    def forward(self, x):
        x = self.features(x)
        x = self.dropout1(x)
        x = self.pool(x).flatten(1)
        x = self.dropout2(x)
        return self.head_emo(x), self.head_gen(x)

# Evaluation util
@torch.no_grad()
def evaluate(model, loader, split_name=""):
    model.eval()
    true_e, pred_e, true_g, pred_g = [], [], [], []
    for imgs, emo_y, gen_y in tqdm(loader, desc=f"Evaluating {split_name}", leave=False):
        imgs = imgs.to(DEVICE); emo_y = emo_y.to(DEVICE); gen_y = gen_y.to(DEVICE)
        emo_logit, gen_logit = model(imgs)
        true_e += emo_y.cpu().tolist()
        pred_e += emo_logit.argmax(1).cpu().tolist()
        true_g += gen_y.cpu().tolist()
        pred_g += gen_logit.argmax(1).cpu().tolist()

    return pd.DataFrame({
        "true_emotion": emo_le.inverse_transform(np.array(true_e)),
        "pred_emotion": emo_le.inverse_transform(np.array(pred_e)),
        "true_gender":  gen_le.inverse_transform(np.array(true_g)),
        "pred_gender":  gen_le.inverse_transform(np.array(pred_g)),
    })

#  Conf‑Matrix helper
def plot_confusion(df, title="Confusion"):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    cm_e = confusion_matrix(df["true_emotion"], df["pred_emotion"],
                            labels=emotion_labels, normalize='true')*100
    sns.heatmap(cm_e, annot=True, fmt=".1f", cmap="Blues",
                xticklabels=emotion_labels, yticklabels=emotion_labels,
                cbar_kws={'label':'%'}, ax=axes[0])
    axes[0].set_title(f"{title} – Emotion"); axes[0].set_xlabel("Pred"); axes[0].set_ylabel("True")

    cm_g = confusion_matrix(df["true_gender"], df["pred_gender"],
                            labels=gender_labels, normalize='true')*100
    sns.heatmap(cm_g, annot=True, fmt=".1f", cmap="Greens",
                xticklabels=gender_labels, yticklabels=gender_labels,
                cbar_kws={'label':'%'}, ax=axes[1])
    axes[1].set_title(f"{title} – Gender"); axes[1].set_xlabel("Pred"); axes[1].set_ylabel("True")

    plt.tight_layout(); plt.show()
    wandb.log({f"{title}_conf_matrix": wandb.Image(fig)})




#
#  Classification reports, curves, ROC plots
#
def print_classification_reports(df, split_name=""):
    print(f"\n {split_name} Emotion Classification Report:")
    print(classification_report(df["true_emotion"],
                                df["pred_emotion"],
                                target_names=emotion_labels))

    print(f"\n {split_name} Gender Classification Report:")
    print(classification_report(df["true_gender"],
                                df["pred_gender"],
                                target_names=gender_labels))




    #  internal helper
def _smooth(arr, win: int = 2, *, do: bool = True):
    """Uniform 1‑D smoothing for nicer curves."""
    if not do or len(arr) < win:
        return np.asarray(arr)
    return snd.uniform_filter1d(np.asarray(arr, dtype=float), size=win, mode="nearest")


# Plot Accuracy Curves
def plot_accuracy_curves(train_emo_accs, val_emo_accs,
                         train_gen_accs,  val_gen_accs,
                         *, smooth: bool = True):
    epochs = np.arange(1, len(train_emo_accs) + 1)
    plt.figure(figsize=(10, 6))

    plt.plot(epochs, _smooth(train_emo_accs, do=smooth), label="Train Emotion Acc",  linewidth=2)
    plt.plot(epochs, _smooth(val_emo_accs,  do=smooth), label="Val  Emotion Acc",   linewidth=2)
    plt.plot(epochs, _smooth(train_gen_accs, do=smooth), "--", label="Train Gender Acc", linewidth=2)
    plt.plot(epochs, _smooth(val_gen_accs,  do=smooth), "--", label="Val  Gender Acc",  linewidth=2)

    plt.title("Training vs Validation Accuracy")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.ylim(0, 1)
    plt.grid(True, alpha=.3); plt.legend(); plt.tight_layout(); plt.show()
    wandb.log({"accuracy_curves": wandb.Image(plt)})


#  Plot Loss Curves
def plot_loss_curves(train_losses, val_losses,
                     train_emo,   val_emo,
                     train_gen,   val_gen,
                     *, smooth: bool = True):
    epochs = np.arange(1, len(train_losses) + 1)
    plt.figure(figsize=(10, 7))

    plt.plot(epochs, _smooth(train_losses, do=smooth), label="Train Total", linewidth=2)
    plt.plot(epochs, _smooth(val_losses,   do=smooth), label="Val  Total",  linewidth=2)

    plt.plot(epochs, _smooth(train_emo, do=smooth), "--", label="Train Emotion")
    plt.plot(epochs, _smooth(val_emo,   do=smooth), "--", label="Val  Emotion")
    plt.plot(epochs, _smooth(train_gen, do=smooth), ":",  label="Train Gender")
    plt.plot(epochs, _smooth(val_gen,   do=smooth), ":",  label="Val  Gender")

    plt.title("Loss Curves")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.grid(True, alpha=.3)
    plt.legend(); plt.tight_layout(); plt.show()
    wandb.log({"loss_curves": wandb.Image(plt)})


#  Collect predictions helper (with optional probabilities)
@torch.no_grad()
def collect_preds(loader, return_probs: bool = False):
    model.eval()
    emo_true, emo_pred, gen_true, gen_pred = [], [], [], []
    emo_probs, gen_probs = [], []

    for imgs, e_y, g_y in loader:
        imgs = imgs.to(DEVICE)
        e_y  = e_y.to(DEVICE)
        g_y  = g_y.to(DEVICE)

        emo_logits, gen_logits = model(imgs)

        emo_true.extend(e_y.cpu().numpy())
        gen_true.extend(g_y.cpu().numpy())
        emo_pred.extend(torch.argmax(emo_logits, 1).cpu().numpy())
        gen_pred.extend(torch.argmax(gen_logits, 1).cpu().numpy())

        if return_probs:
            emo_probs.append(torch.softmax(emo_logits, 1).cpu())
            gen_probs.append(torch.softmax(gen_logits, 1).cpu())

    if return_probs:
        return (
            np.array(emo_true), np.array(emo_pred),
            np.array(gen_true), np.array(gen_pred),
            torch.cat(emo_probs).numpy(), torch.cat(gen_probs).numpy()
        )
    else:
        return (
            np.array(emo_true), np.array(emo_pred),
            np.array(gen_true), np.array(gen_pred)
        )

# ensure gender prob arrays have two columns (binary case)
def _ensure_two_cols(arr):
    """If arr is shape [N] or [N,1], convert to [N,2] as [1-p, p]."""
    arr = np.asarray(arr)
    if arr.ndim == 1 or (arr.ndim == 2 and arr.shape[1] == 1):
        arr = arr.reshape(-1, 1)
        # Assuming the single column is the probability for the second class (Woman)
        # You might need to adjust this based on your LabelEncoder mapping
        arr = np.hstack([(1.0 - arr), arr])
    return arr


#Extended ROC Curve Plot
def plot_extended_roc(split_name, true_emo, prob_emo, true_gen, prob_gen):
    true_emo_bin = label_binarize(true_emo, classes=list(range(len(emotion_labels))))
    true_gen_bin = label_binarize(true_gen, classes=list(range(len(gender_labels))))

    #  Emotion ROC
    fpr_e, tpr_e, roc_auc_e = {}, {}, {}
    for i in range(len(emotion_labels)):
        fpr_e[i], tpr_e[i], _ = roc_curve(true_emo_bin[:, i], prob_emo[:, i])
        roc_auc_e[i] = auc(fpr_e[i], tpr_e[i])

    fpr_e["micro"], tpr_e["micro"], _ = roc_curve(true_emo_bin.ravel(), prob_emo.ravel())
    roc_auc_e["micro"] = auc(fpr_e["micro"], tpr_e["micro"])

    all_fpr = np.unique(np.concatenate([fpr_e[i] for i in range(len(emotion_labels))]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(len(emotion_labels)):
        mean_tpr += np.interp(all_fpr, fpr_e[i], tpr_e[i])
    mean_tpr /= len(emotion_labels)
    fpr_e["macro"], tpr_e["macro"] = all_fpr, mean_tpr
    roc_auc_e["macro"] = auc(fpr_e["macro"], tpr_e["macro"])

    plt.figure(figsize=(10, 7))
    plt.plot(fpr_e["micro"], tpr_e["micro"],
             linestyle='--', label=f"Micro‑avg (AUC = {roc_auc_e['micro']:.2f})")
    plt.plot(fpr_e["macro"], tpr_e["macro"],
             label=f"Macro‑avg (AUC = {roc_auc_e['macro']:.2f})")
    colors = cycle(sns.color_palette("hls", len(emotion_labels)))
    for i, c in zip(range(len(emotion_labels)), colors):
        plt.plot(fpr_e[i], tpr_e[i], color=c,
                 label=f"{emotion_labels[i]} (AUC = {roc_auc_e[i]:.2f})")

    plt.plot([0, 1], [0, 1], 'k--'); plt.title(f"{split_name} Emotion ROC")
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(fontsize="small"); plt.grid(True)
    plt.tight_layout(); plt.show()
    wandb.log({f"roc/{split_name.lower()}_emotion": wandb.Image(plt)})

    #  Gender ROC
    # Ensure gender probabilities have two columns
    prob_gen = _ensure_two_cols(prob_gen)

    fpr_g, tpr_g, roc_auc_g = {}, {}, {}
    for i in range(len(gender_labels)):
        fpr_g[i], tpr_g[i], _ = roc_curve(true_gen_bin[:, i], prob_gen[:, i])
        roc_auc_g[i] = auc(fpr_g[i], tpr_g[i])
    fpr_g["micro"], tpr_g["micro"], _ = roc_curve(true_gen_bin.ravel(), prob_gen.ravel())
    roc_auc_g["micro"] = auc(fpr_g["micro"], tpr_g["micro"])

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_g["micro"], tpr_g["micro"],
             linestyle='--', label=f"Micro‑avg (AUC = {roc_auc_g['micro']:.2f})")
    for i in range(len(gender_labels)):
        plt.plot(fpr_g[i], tpr_g[i], label=f"{gender_labels[i]} (AUC = {roc_auc_g[i]:.2f})")
    plt.plot([0, 1], [0, 1], 'k--'); plt.title(f"{split_name} Gender ROC")
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(); plt.grid(True)
    plt.tight_layout(); plt.show()
    wandb.log({f"roc/{split_name.lower()}_gender": wandb.Image(plt)})


# Fairness Audit (gender × emotion)
def fairness_audit(df):
    print("\n🎯 Accuracy by Gender × Emotion:")
    table = (
        df.groupby(["true_gender", "true_emotion"])
          .apply(lambda g: (g["true_emotion"] == g["pred_emotion"]).mean())
          .unstack().fillna(0)
    )
    print(table.round(3))

    for g in table.index:
        for e in table.columns:
            wandb.log({f"fairness/{g}/{e}": table.loc[g, e]})

    pos_rate = df["pred_gender"].value_counts(normalize=True)
    disp_impact = pos_rate.get("Woman", 0) / pos_rate.get("Man", 1e-6)
    wandb.log({"disparate_impact_ratio": disp_impact})

    equal_opp = table.loc["Woman"].mean() - table.loc["Man"].mean()
    wandb.log({"equal_opportunity_diff": equal_opp})

    table.T.plot(kind="bar", figsize=(10, 6), ylim=(0, 1))
    plt.title("Emotion Accuracy by Gender"); plt.ylabel("Accuracy")
    plt.grid(axis="y"); plt.tight_layout(); plt.show()
    wandb.log({"gender_bias_bar": wandb.Image(plt)})


#  Model Initialization
model = MultiTaskCNN().to(DEVICE)

#  Optimizer, Scheduler, Loss
#initial_lr = 1e-3
#opt = torch.optim.Adam(model.parameters(), lr=initial_lr, weight_decay=1e-4)

# Smarter scheduler
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=3, gamma=0.5)

initial_lr = 3e-4
opt = torch.optim.Adam(model.parameters(), lr=initial_lr, weight_decay=1e-4)

# linear warmup first 3 epochs then cosine
def lr_lambda(cur_epoch):
    warmup = 3
    if cur_epoch < warmup:
        return (cur_epoch + 1) / warmup
    # cosine from warmup to EPOCHS
    t = (cur_epoch - warmup) / max(1, (EPOCHS - warmup))
    return 0.5 * (1 + np.cos(np.pi * t))

scheduler = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)

# Label-smoothed loss for better generalization
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

#  Tracking
train_losses = []
val_losses = []
train_emo_losses = []
train_gen_losses = []
val_emo_losses = []
val_gen_losses = []

train_emo_accuracies = []
train_gen_accuracies = []
val_emo_accuracies = []
val_gen_accuracies = []

# Early stopping setup
patience = 5
min_delta = 1e-3
best_val_loss = np.inf
best_val_metric = 0
best_epoch = 0
no_imp = 0

#  Training Loop
for epoch in range(EPOCHS):
    print(f"\n Epoch {epoch+1}/{EPOCHS}")

    model.train()
    tl = train_loader
    total_loss = total_emo_loss = total_gen_loss = 0
    correct_emo = correct_gen = total_emo = total_gen = 0

    for imgs, emo_y, gen_y in tqdm(tl, desc="Train", leave=False):
        imgs, emo_y, gen_y = imgs.to(DEVICE), emo_y.to(DEVICE), gen_y.to(DEVICE)
        emo_logit, gen_logit = model(imgs)

        loss_e = criterion(emo_logit, emo_y)
        loss_g = criterion(gen_logit, gen_y)
        loss = 0.7 * loss_e + 0.3 * loss_g

        opt.zero_grad()
        loss.backward()
        opt.step()

        bs = imgs.size(0)
        total_loss     += loss.item()
        total_emo_loss += loss_e.item()
        total_gen_loss += loss_g.item()
        correct_emo    += (emo_logit.argmax(1) == emo_y).sum().item()
        correct_gen    += (gen_logit.argmax(1) == gen_y).sum().item()
        total_emo      += bs
        total_gen      += bs

    train_losses.append(total_loss / len(tl))
    train_emo_losses.append(total_emo_loss / len(tl))
    train_gen_losses.append(total_gen_loss / len(tl))
    train_emo_accuracies.append(correct_emo / total_emo)
    train_gen_accuracies.append(correct_gen / total_gen)

    #  Validation
    model.eval()
    vl = val_loader
    val_loss_total = val_emo_loss = val_gen_loss = 0
    val_correct_emo = val_correct_gen = val_total_emo = val_total_gen = 0

    with torch.no_grad():
        for imgs, emo_y, gen_y in vl:
            imgs, emo_y, gen_y = imgs.to(DEVICE), emo_y.to(DEVICE), gen_y.to(DEVICE)
            emo_logit, gen_logit = model(imgs)

            loss_e = criterion(emo_logit, emo_y)
            loss_g = criterion(gen_logit, gen_y)
            val_loss_total += (loss_e + loss_g).item()
            val_emo_loss   += loss_e.item()
            val_gen_loss   += loss_g.item()
            val_correct_emo += (emo_logit.argmax(1) == emo_y).sum().item()
            val_correct_gen += (gen_logit.argmax(1) == gen_y).sum().item()
            val_total_emo  += imgs.size(0)
            val_total_gen  += imgs.size(0)

    val_losses.append(val_loss_total / len(vl))
    val_emo_losses.append(val_emo_loss / len(vl))
    val_gen_losses.append(val_gen_loss / len(vl))
    val_emo_accuracies.append(val_correct_emo / val_total_emo)
    val_gen_accuracies.append(val_correct_gen / val_total_gen)

    print(f" Train loss {train_losses[-1]:.4f}, Val loss {val_losses[-1]:.4f}")

    wandb.log({
        "epoch": epoch+1,
        "train_loss": train_losses[-1],
        "val_loss":   val_losses[-1],
        "train_emo_acc": train_emo_accuracies[-1],
        "val_emo_acc":   val_emo_accuracies[-1],
        "train_gen_acc": train_gen_accuracies[-1],
        "val_gen_acc":   val_gen_accuracies[-1],
        "lr": opt.param_groups[0]['lr'],
    })

    scheduler.step()

    # Hybrid Early‑stopping (Loss + Accuracy) #
    val_metric = (val_emo_accuracies[-1] + val_gen_accuracies[-1]) / 2
    loss_improved = val_losses[-1] < best_val_loss - min_delta
    acc_improved  = val_metric     > best_val_metric + min_delta

    if loss_improved or acc_improved:
        best_val_loss = min(best_val_loss, val_losses[-1])
        best_val_metric = max(best_val_metric, val_metric)
        best_epoch = epoch
        no_imp = 0
        torch.save(model.state_dict(), MODEL_PATH)
        print(" Model improved, checkpoint saved.")
    else:
        no_imp += 1
        print(f"No improvement for {no_imp} epoch(s).")
        if no_imp >= patience:
            print("Early‑stopping triggered.")
            break


#  Final Eval
model.load_state_dict(torch.load(MODEL_PATH))
train_df = evaluate(model, train_loader, "train")
val_df   = evaluate(model, val_loader, "val")
test_df  = evaluate(model, test_loader, "test")
test_df.to_csv(PRED_CSV, index=False)
print("Predictions saved.")

plot_confusion(train_df, "Train")
plot_confusion(val_df, "Val")
plot_confusion(test_df, "Test")

print_classification_reports(train_df, "Train")
print_classification_reports(val_df,   "Validation")
print_classification_reports(test_df,  "Test")


plot_accuracy_curves(
    train_emo_accuracies, val_emo_accuracies,
    train_gen_accuracies, val_gen_accuracies
)
plot_loss_curves(
    train_losses, val_losses,
    train_emo_losses, val_emo_losses,
    train_gen_losses, val_gen_losses
)

et_e_t, ep_t, gt_t, gp_t, pro_e_t, pro_g_t = collect_preds(test_loader, return_probs=True)
plot_extended_roc("Test", et_e_t, pro_e_t, gt_t, pro_g_t)

fairness_audit(test_df)

wandb.finish()

**EfficientNet_B2 Model Training (Change data paths for each dataset**)

In [ ]:

# Multitask Emotion & Gender Classification with EfficientNet_B2, Fairness, and wandb ===
#

import os, cv2, io, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm
from itertools import cycle
from PIL import Image

from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_curve, auc)
from sklearn.preprocessing import LabelEncoder, label_binarize
import scipy.ndimage as snd

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import efficientnet_b2
from torch.optim.lr_scheduler import StepLR

import torch
from torch.utils.data import DataLoader
from torchvision import transforms

import wandb
wandb.init(project="multitask-gender-emotion", name="Celeb_A-Scratch‑EfficentNet_B2")

# Config
emotion_labels  = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]
gender_labels   = ["Man", "Woman"]

IMG_SIZE  = 224
BATCH_SIZE = 32
EPOCHS   = 50
LR = 1e-3

CSV_PATH      = "/content/dataset/CK+/processed/CKPLUS_Features_20250805_111308_faceprefixed_CLEAN.csv"
TRAIN_IMG_DIR = "/content/dataset/CK+/processed/train"
VAL_IMG_DIR   = "/content/dataset/CK+/processed/val"
TEST_IMG_DIR  = "/content/dataset/CK+/processed/test"


MODEL_PATH = "multitask_model_Scratch_efficientnetb2.pt"
PRED_CSV   = "multitask_predictions_Sratch_efficientnetb2.csv"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

emo_le = LabelEncoder().fit(emotion_labels)
gen_le = LabelEncoder().fit(gender_labels)





from torchvision import transforms

#  Optimized Train Transform for CK+ Dataset
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.90, 1.0)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(8),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

#  Evaluation Transform (No Augmentation)
eval_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



# Your Dataset Class (unchanged)
class AffectNetDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, verbose=True):
        self.df, self.img_dir, self.transform, self.verbose = df.reset_index(drop=True), img_dir, transform, verbose
        print(f"CKplus → {len(self.df)} samples from {self.img_dir}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row["filename"])

        if self.verbose and idx % 1000 == 0:
            print(f"[INFO] Loading {idx}/{len(self.df)}: {path}")

        img = cv2.imread(path)
        if img is None:
            img = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        img = self.transform(img) if self.transform else transforms.ToTensor()(img / 255.)

        emo = torch.tensor(emo_le.transform([row["dominant_emotion"]])[0])
        gen = torch.tensor(gen_le.transform([row["gender"]])[0])
        return img, emo, gen

#  Your make_loader Function
def make_loader(df, img_dir, *, shuffle, train):
    tfm = train_transform if train else eval_transform
    return DataLoader(
        AffectNetDataset(df, img_dir, transform=tfm),
        batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=0
    )


#  New: Create all loaders
def create_dataloaders(csv_path=CSV_PATH,
                       train_img_dir=TRAIN_IMG_DIR,
                       val_img_dir=VAL_IMG_DIR,
                       test_img_dir=TEST_IMG_DIR,
                       batch_size=BATCH_SIZE):
    # Load full CSV
    df = pd.read_csv(csv_path)

    # Split df by your 'split' column (make sure your CSV has this)
    train_df = df[df['dataset'] == 'train'].reset_index(drop=True)
    val_df = df[df['dataset'] == 'val'].reset_index(drop=True)
    test_df = df[df['dataset'] == 'test'].reset_index(drop=True)

    # Create loaders using your make_loader func and existing transforms
    train_loader = make_loader(train_df, train_img_dir, shuffle=True, train=True)
    val_loader = make_loader(val_df, val_img_dir, shuffle=False, train=False)
    test_loader = make_loader(test_df, test_img_dir, shuffle=False, train=False)

    return train_loader, val_loader, test_loader


def mixup_data(x, y1, y2, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y1_a, y1_b = y1, y1[index]
    y2_a, y2_b = y2, y2[index]
    return mixed_x, y1_a, y1_b, y2_a, y2_b, lam


#  Example of usage
train_loader, val_loader, test_loader = create_dataloaders()


#  Multi‑Task EfficientNet‑B2
class MultiTaskCNN(nn.Module):  # same class name for compatibility
    def __init__(self):
        super().__init__()
        backbone = efficientnet_b2(weights=None)
        self.features = backbone.features
        self.pool = nn.AdaptiveAvgPool2d(1)
        in_feats = backbone.classifier[1].in_features  # usually 1408 for B2
        self.dropout = nn.Dropout(0.3)
        self.head_emo = nn.Linear(in_feats, len(emotion_labels))
        self.head_gen = nn.Linear(in_feats, len(gender_labels))

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x).flatten(1)
        x = self.dropout(x)
        return self.head_emo(x), self.head_gen(x)

# Evaluation util
@torch.no_grad()
def evaluate(model, loader, split_name=""):
    model.eval()
    true_e, pred_e, true_g, pred_g = [], [], [], []
    for imgs, emo_y, gen_y in tqdm(loader, desc=f"Evaluating {split_name}", leave=False):
        imgs = imgs.to(DEVICE); emo_y = emo_y.to(DEVICE); gen_y = gen_y.to(DEVICE)
        emo_logit, gen_logit = model(imgs)
        true_e += emo_y.cpu().tolist()
        pred_e += emo_logit.argmax(1).cpu().tolist()
        true_g += gen_y.cpu().tolist()
        pred_g += gen_logit.argmax(1).cpu().tolist()

    return pd.DataFrame({
        "true_emotion": emo_le.inverse_transform(np.array(true_e)),
        "pred_emotion": emo_le.inverse_transform(np.array(pred_e)),
        "true_gender":  gen_le.inverse_transform(np.array(true_g)),
        "pred_gender":  gen_le.inverse_transform(np.array(pred_g)),
    })

#  Conf‑Matrix helper
def plot_confusion(df, title="Confusion"):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    cm_e = confusion_matrix(df["true_emotion"], df["pred_emotion"],
                            labels=emotion_labels, normalize='true')*100
    sns.heatmap(cm_e, annot=True, fmt=".1f", cmap="Blues",
                xticklabels=emotion_labels, yticklabels=emotion_labels,
                cbar_kws={'label':'%'}, ax=axes[0])
    axes[0].set_title(f"{title} – Emotion"); axes[0].set_xlabel("Pred"); axes[0].set_ylabel("True")

    cm_g = confusion_matrix(df["true_gender"], df["pred_gender"],
                            labels=gender_labels, normalize='true')*100
    sns.heatmap(cm_g, annot=True, fmt=".1f", cmap="Greens",
                xticklabels=gender_labels, yticklabels=gender_labels,
                cbar_kws={'label':'%'}, ax=axes[1])
    axes[1].set_title(f"{title} – Gender"); axes[1].set_xlabel("Pred"); axes[1].set_ylabel("True")

    plt.tight_layout(); plt.show()
    wandb.log({f"{title}_conf_matrix": wandb.Image(fig)})




#
#  Classification reports, curves, ROC plots
#
def print_classification_reports(df, split_name=""):
    print(f"\n {split_name} Emotion Classification Report:")
    print(classification_report(df["true_emotion"],
                                df["pred_emotion"],
                                target_names=emotion_labels))

    print(f"\n {split_name} Gender Classification Report:")
    print(classification_report(df["true_gender"],
                                df["pred_gender"],
                                target_names=gender_labels))




    #  internal helper
def _smooth(arr, win: int = 2, *, do: bool = True):
    """Uniform 1‑D smoothing for nicer curves."""
    if not do or len(arr) < win:
        return np.asarray(arr)
    return snd.uniform_filter1d(np.asarray(arr, dtype=float), size=win, mode="nearest")


# Plot Accuracy Curves
def plot_accuracy_curves(train_emo_accs, val_emo_accs,
                         train_gen_accs,  val_gen_accs,
                         *, smooth: bool = True):
    epochs = np.arange(1, len(train_emo_accs) + 1)
    plt.figure(figsize=(10, 6))

    plt.plot(epochs, _smooth(train_emo_accs, do=smooth), label="Train Emotion Acc",  linewidth=2)
    plt.plot(epochs, _smooth(val_emo_accs,  do=smooth), label="Val  Emotion Acc",   linewidth=2)
    plt.plot(epochs, _smooth(train_gen_accs, do=smooth), "--", label="Train Gender Acc", linewidth=2)
    plt.plot(epochs, _smooth(val_gen_accs,  do=smooth), "--", label="Val  Gender Acc",  linewidth=2)

    plt.title("Training vs Validation Accuracy")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.ylim(0, 1)
    plt.grid(True, alpha=.3); plt.legend(); plt.tight_layout(); plt.show()
    wandb.log({"accuracy_curves": wandb.Image(plt)})


#  Plot Loss Curves
def plot_loss_curves(train_losses, val_losses,
                     train_emo,   val_emo,
                     train_gen,   val_gen,
                     *, smooth: bool = True):
    epochs = np.arange(1, len(train_losses) + 1)
    plt.figure(figsize=(10, 7))

    plt.plot(epochs, _smooth(train_losses, do=smooth), label="Train Total", linewidth=2)
    plt.plot(epochs, _smooth(val_losses,   do=smooth), label="Val  Total",  linewidth=2)

    plt.plot(epochs, _smooth(train_emo, do=smooth), "--", label="Train Emotion")
    plt.plot(epochs, _smooth(val_emo,   do=smooth), "--", label="Val  Emotion")
    plt.plot(epochs, _smooth(train_gen, do=smooth), ":",  label="Train Gender")
    plt.plot(epochs, _smooth(val_gen,   do=smooth), ":",  label="Val  Gender")

    plt.title("Loss Curves")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.grid(True, alpha=.3)
    plt.legend(); plt.tight_layout(); plt.show()
    wandb.log({"loss_curves": wandb.Image(plt)})


#  Collect predictions helper (with optional probabilities)
@torch.no_grad()
def collect_preds(loader, return_probs: bool = False):
    model.eval()
    emo_true, emo_pred, gen_true, gen_pred = [], [], [], []
    emo_probs, gen_probs = [], []

    for imgs, e_y, g_y in loader:
        imgs = imgs.to(DEVICE)
        e_y  = e_y.to(DEVICE)
        g_y  = g_y.to(DEVICE)

        emo_logits, gen_logits = model(imgs)

        emo_true.extend(e_y.cpu().numpy())
        gen_true.extend(g_y.cpu().numpy())
        emo_pred.extend(torch.argmax(emo_logits, 1).cpu().numpy())
        gen_pred.extend(torch.argmax(gen_logits, 1).cpu().numpy())

        if return_probs:
            emo_probs.append(torch.softmax(emo_logits, 1).cpu())
            gen_probs.append(torch.softmax(gen_logits, 1).cpu())

    if return_probs:
        return (
            np.array(emo_true), np.array(emo_pred),
            np.array(gen_true), np.array(gen_pred),
            torch.cat(emo_probs).numpy(), torch.cat(gen_probs).numpy()
        )
    else:
        return (
            np.array(emo_true), np.array(emo_pred),
            np.array(gen_true), np.array(gen_pred)
        )

# ensure gender prob arrays have two columns (binary case)
def _ensure_two_cols(arr):
    """If arr is shape [N] or [N,1], convert to [N,2] as [1-p, p]."""
    arr = np.asarray(arr)
    if arr.ndim == 1 or (arr.ndim == 2 and arr.shape[1] == 1):
        arr = arr.reshape(-1, 1)
        # Assuming the single column is the probability for the second class (Woman)
        # You might need to adjust this based on your LabelEncoder mapping
        arr = np.hstack([(1.0 - arr), arr])
    return arr


#  Extended ROC Curve Plot
def plot_extended_roc(split_name, true_emo, prob_emo, true_gen, prob_gen):
    true_emo_bin = label_binarize(true_emo, classes=list(range(len(emotion_labels))))
    true_gen_bin = label_binarize(true_gen, classes=list(range(len(gender_labels))))

    # Emotion ROC
    fpr_e, tpr_e, roc_auc_e = {}, {}, {}
    for i in range(len(emotion_labels)):
        fpr_e[i], tpr_e[i], _ = roc_curve(true_emo_bin[:, i], prob_emo[:, i])
        roc_auc_e[i] = auc(fpr_e[i], tpr_e[i])

    fpr_e["micro"], tpr_e["micro"], _ = roc_curve(true_emo_bin.ravel(), prob_emo.ravel())
    roc_auc_e["micro"] = auc(fpr_e["micro"], tpr_e["micro"])

    all_fpr = np.unique(np.concatenate([fpr_e[i] for i in range(len(emotion_labels))]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(len(emotion_labels)):
        mean_tpr += np.interp(all_fpr, fpr_e[i], tpr_e[i])
    mean_tpr /= len(emotion_labels)
    fpr_e["macro"], tpr_e["macro"] = all_fpr, mean_tpr
    roc_auc_e["macro"] = auc(fpr_e["macro"], tpr_e["macro"])

    plt.figure(figsize=(10, 7))
    plt.plot(fpr_e["micro"], tpr_e["micro"],
             linestyle='--', label=f"Micro‑avg (AUC = {roc_auc_e['micro']:.2f})")
    plt.plot(fpr_e["macro"], tpr_e["macro"],
             label=f"Macro‑avg (AUC = {roc_auc_e['macro']:.2f})")
    colors = cycle(sns.color_palette("hls", len(emotion_labels)))
    for i, c in zip(range(len(emotion_labels)), colors):
        plt.plot(fpr_e[i], tpr_e[i], color=c,
                 label=f"{emotion_labels[i]} (AUC = {roc_auc_e[i]:.2f})")

    plt.plot([0, 1], [0, 1], 'k--'); plt.title(f"{split_name} Emotion ROC")
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(fontsize="small"); plt.grid(True)
    plt.tight_layout(); plt.show()
    wandb.log({f"roc/{split_name.lower()}_emotion": wandb.Image(plt)})

    # Gender ROC
    # Ensure gender probabilities have two columns
    prob_gen = _ensure_two_cols(prob_gen)

    fpr_g, tpr_g, roc_auc_g = {}, {}, {}
    for i in range(len(gender_labels)):
        fpr_g[i], tpr_g[i], _ = roc_curve(true_gen_bin[:, i], prob_gen[:, i])
        roc_auc_g[i] = auc(fpr_g[i], tpr_g[i])
    fpr_g["micro"], tpr_g["micro"], _ = roc_curve(true_gen_bin.ravel(), prob_gen.ravel())
    roc_auc_g["micro"] = auc(fpr_g["micro"], tpr_g["micro"])

    plt.figure(figsize=(8, 6))
    plt.plot(fpr_g["micro"], tpr_g["micro"],
             linestyle='--', label=f"Micro‑avg (AUC = {roc_auc_g['micro']:.2f})")
    for i in range(len(gender_labels)):
        plt.plot(fpr_g[i], tpr_g[i], label=f"{gender_labels[i]} (AUC = {roc_auc_g[i]:.2f})")
    plt.plot([0, 1], [0, 1], 'k--'); plt.title(f"{split_name} Gender ROC")
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(); plt.grid(True)
    plt.tight_layout(); plt.show()
    wandb.log({f"roc/{split_name.lower()}_gender": wandb.Image(plt)})


# Fairness Audit (gender × emotion)
def fairness_audit(df):
    print("\n🎯 Accuracy by Gender × Emotion:")
    table = (
        df.groupby(["true_gender", "true_emotion"])
          .apply(lambda g: (g["true_emotion"] == g["pred_emotion"]).mean())
          .unstack().fillna(0)
    )
    print(table.round(3))

    for g in table.index:
        for e in table.columns:
            wandb.log({f"fairness/{g}/{e}": table.loc[g, e]})

    pos_rate = df["pred_gender"].value_counts(normalize=True)
    disp_impact = pos_rate.get("Woman", 0) / pos_rate.get("Man", 1e-6)
    wandb.log({"disparate_impact_ratio": disp_impact})

    equal_opp = table.loc["Woman"].mean() - table.loc["Man"].mean()
    wandb.log({"equal_opportunity_diff": equal_opp})

    table.T.plot(kind="bar", figsize=(10, 6), ylim=(0, 1))
    plt.title("Emotion Accuracy by Gender"); plt.ylabel("Accuracy")
    plt.grid(axis="y"); plt.tight_layout(); plt.show()
    wandb.log({"gender_bias_bar": wandb.Image(plt)})


#  Model Initialization
model = MultiTaskCNN().to(DEVICE)

#  Optimizer, Scheduler, Loss
#initial_lr = 1e-3
#opt = torch.optim.Adam(model.parameters(), lr=initial_lr, weight_decay=1e-4)

# Smarter scheduler
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=3, gamma=0.5)

initial_lr = 3e-4
opt = torch.optim.Adam(model.parameters(), lr=initial_lr, weight_decay=1e-4)

# linear warmup first 3 epochs then cosine
def lr_lambda(cur_epoch):
    warmup = 3
    if cur_epoch < warmup:
        return (cur_epoch + 1) / warmup
    # cosine from warmup to EPOCHS
    t = (cur_epoch - warmup) / max(1, (EPOCHS - warmup))
    return 0.5 * (1 + np.cos(np.pi * t))

scheduler = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)

# Label-smoothed loss for better generalization
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

#  Tracking
train_losses = []
val_losses = []
train_emo_losses = []
train_gen_losses = []
val_emo_losses = []
val_gen_losses = []

train_emo_accuracies = []
train_gen_accuracies = []
val_emo_accuracies = []
val_gen_accuracies = []

# Early stopping setup
patience = 5
min_delta = 1e-3
best_val_loss = np.inf
best_val_metric = 0
best_epoch = 0
no_imp = 0

#  Training Loop
for epoch in range(EPOCHS):
    print(f"\n Epoch {epoch+1}/{EPOCHS}")

    model.train()
    tl = train_loader
    total_loss = total_emo_loss = total_gen_loss = 0
    correct_emo = correct_gen = total_emo = total_gen = 0

    for imgs, emo_y, gen_y in tqdm(tl, desc="Train", leave=False):
        imgs, emo_y, gen_y = imgs.to(DEVICE), emo_y.to(DEVICE), gen_y.to(DEVICE)
        emo_logit, gen_logit = model(imgs)

        loss_e = criterion(emo_logit, emo_y)
        loss_g = criterion(gen_logit, gen_y)
        loss = 0.7 * loss_e + 0.3 * loss_g

        opt.zero_grad()
        loss.backward()
        opt.step()

        bs = imgs.size(0)
        total_loss     += loss.item()
        total_emo_loss += loss_e.item()
        total_gen_loss += loss_g.item()
        correct_emo    += (emo_logit.argmax(1) == emo_y).sum().item()
        correct_gen    += (gen_logit.argmax(1) == gen_y).sum().item()
        total_emo      += bs
        total_gen      += bs

    train_losses.append(total_loss / len(tl))
    train_emo_losses.append(total_emo_loss / len(tl))
    train_gen_losses.append(total_gen_loss / len(tl))
    train_emo_accuracies.append(correct_emo / total_emo)
    train_gen_accuracies.append(correct_gen / total_gen)

    #  Validation
    model.eval()
    vl = val_loader
    val_loss_total = val_emo_loss = val_gen_loss = 0
    val_correct_emo = val_correct_gen = val_total_emo = val_total_gen = 0

    with torch.no_grad():
        for imgs, emo_y, gen_y in vl:
            imgs, emo_y, gen_y = imgs.to(DEVICE), emo_y.to(DEVICE), gen_y.to(DEVICE)
            emo_logit, gen_logit = model(imgs)

            loss_e = criterion(emo_logit, emo_y)
            loss_g = criterion(gen_logit, gen_y)
            val_loss_total += (loss_e + loss_g).item()
            val_emo_loss   += loss_e.item()
            val_gen_loss   += loss_g.item()
            val_correct_emo += (emo_logit.argmax(1) == emo_y).sum().item()
            val_correct_gen += (gen_logit.argmax(1) == gen_y).sum().item()
            val_total_emo  += imgs.size(0)
            val_total_gen  += imgs.size(0)

    val_losses.append(val_loss_total / len(vl))
    val_emo_losses.append(val_emo_loss / len(vl))
    val_gen_losses.append(val_gen_loss / len(vl))
    val_emo_accuracies.append(val_correct_emo / val_total_emo)
    val_gen_accuracies.append(val_correct_gen / val_total_gen)

    print(f" Train loss {train_losses[-1]:.4f}, Val loss {val_losses[-1]:.4f}")

    wandb.log({
        "epoch": epoch+1,
        "train_loss": train_losses[-1],
        "val_loss":   val_losses[-1],
        "train_emo_acc": train_emo_accuracies[-1],
        "val_emo_acc":   val_emo_accuracies[-1],
        "train_gen_acc": train_gen_accuracies[-1],
        "val_gen_acc":   val_gen_accuracies[-1],
        "lr": opt.param_groups[0]['lr'],
    })

    scheduler.step()

    # Hybrid Early‑stopping (Loss + Accuracy) #
    val_metric = (val_emo_accuracies[-1] + val_gen_accuracies[-1]) / 2
    loss_improved = val_losses[-1] < best_val_loss - min_delta
    acc_improved  = val_metric     > best_val_metric + min_delta

    if loss_improved or acc_improved:
        best_val_loss = min(best_val_loss, val_losses[-1])
        best_val_metric = max(best_val_metric, val_metric)
        best_epoch = epoch
        no_imp = 0
        torch.save(model.state_dict(), MODEL_PATH)
        print(" Model improved, checkpoint saved.")
    else:
        no_imp += 1
        print(f"No improvement for {no_imp} epoch(s).")
        if no_imp >= patience:
            print("Early‑stopping triggered.")
            break


#  Final Eval
model.load_state_dict(torch.load(MODEL_PATH))
train_df = evaluate(model, train_loader, "train")
val_df   = evaluate(model, val_loader, "val")
test_df  = evaluate(model, test_loader, "test")
test_df.to_csv(PRED_CSV, index=False)
print("Predictions saved.")

plot_confusion(train_df, "Train")
plot_confusion(val_df, "Val")
plot_confusion(test_df, "Test")

print_classification_reports(train_df, "Train")
print_classification_reports(val_df,   "Validation")
print_classification_reports(test_df,  "Test")


plot_accuracy_curves(
    train_emo_accuracies, val_emo_accuracies,
    train_gen_accuracies, val_gen_accuracies
)
plot_loss_curves(
    train_losses, val_losses,
    train_emo_losses, val_emo_losses,
    train_gen_losses, val_gen_losses
)

et_e_t, ep_t, gt_t, gp_t, pro_e_t, pro_g_t = collect_preds(test_loader, return_probs=True)
plot_extended_roc("Test", et_e_t, pro_e_t, gt_t, pro_g_t)

fairness_audit(test_df)

wandb.finish()